In [18]:
import tqdm
import torch, cv2
import numpy as np
import matplotlib.pyplot as plt

from PIL import Image
from torchvision.ops import box_iou
import PIL, cv2
import sys
sys.path.append('../scripts/')
from resize_boxes import resize_values
%load_ext autoreload
%autoreload 2
from tqdm import tqdm
from sklearn.cluster import KMeans

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
image_paths = np.loadtxt('../../bird_dataset/CUB_200_2011/images.txt', dtype=str) # all image paths
bboxes_file_path = "../../bird_dataset/CUB_200_2011/bounding_boxes.txt"
bboxes = (np.loadtxt(bboxes_file_path, dtype=int, delimiter=" ", ndmin=2)[:, 1:]) # Take only x, y, w, h 
# bboxes[:, 2] = bboxes[:, 2] + bboxes[:, 0] # get corner point
# bboxes[:, 3] = bboxes[:, 3] + bboxes[:, 1] # get corner point

/tmp/ipykernel_16388/979166117.py:3: DeprecationWarning: loadtxt(): Parsing an integer via a float is deprecated.  To avoid this warning, you can:
    * make sure the original data is stored as integers.
    * use the `converters=` keyword argument.  If you only use
      NumPy 1.23 or later, `converters=float` will normally work.
    * Use `np.loadtxt(...).astype(np.int64)` parsing the file as
      floating point and then convert it.  (On all NumPy versions.)
  (Deprecated NumPy 1.23)
  bboxes = (np.loadtxt(bboxes_file_path, dtype=int, delimiter=" ", ndmin=2)[:, 1:]) # Take only x, y, w, h


In [10]:
updated_boxes = []
for x in tqdm(range(len(image_paths))):
    img = np.array(PIL.Image.open('../../bird_dataset/CUB_200_2011/images/' + image_paths[x][1]))
    updated_boxes.append(resize_values(bboxes[x], img.shape, (512, 512), return_old_box=False, is_torch=False))

  0%|          | 0/11788 [00:00<?, ?it/s]

100%|██████████| 11788/11788 [00:28<00:00, 416.80it/s]


In [11]:
updated_boxes = torch.Tensor(updated_boxes).float()

In [13]:
def IoU(clusters: torch.tensor, bboxes: torch.tensor):
    iou_values = box_iou(clusters, bboxes)
    return iou_values

In [26]:
def KMeans(bboxes:torch.tensor, k:int, dist=torch.mean, stop_iter=5):
    rows = bboxes.shape[0]
    distances = torch.empty((rows, k))
    last_clusters = torch.zeros((rows, ))

    cluster_indxs = np.random.choice(rows, k, replace=False) # choose unique indexs in rows
    clusters = bboxes[cluster_indxs].clone()

    iteration = 0
    while True:
        # calculate the distances 
        distances = IoU(bboxes, clusters)

        nearest_clusters = torch.argmax(distances, dim=1) # 0, 1, 2 ... K   

        if (last_clusters == nearest_clusters).all(): # break if nothing changes
            iteration += 1
            if iteration == stop_iter:
                break
        else:
            iteration = 0
        # Take the mean and step for cluster coordiantes 
        for cluster in range(k):
            clusters[cluster] = torch.mean(bboxes[nearest_clusters == cluster], axis=0)

        last_clusters = nearest_clusters.clone()
    return clusters, distances

In [186]:
clusterList = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
mean_distances, anchorBoxes = [], []

for cluster_k in clusterList:
    anchors, distances = KMeans(updated_boxes, k=cluster_k)
    indxs = torch.argmax(distances, dim=1)
    filtered_distances = []
    for i, distance in enumerate(distances):
        filtered_distances.append(distance[indxs[i]].item())
    mean_distances.append(np.mean(filtered_distances))
    anchorBoxes.append(anchors)
    
plt.plot(clusterList, mean_distances)
plt.scatter(clusterList, mean_distances)
plt.title("Mean IoU Score")
plt.xlabel("Number of clusters")
plt.ylabel("IoU score")

KeyboardInterrupt: 

In [40]:
anchors, distances = KMeans(updated_boxes[:100], k=3)

KeyboardInterrupt: 

In [37]:
anchors

tensor([[128.1476, 118.9425, 127.1325, 176.1158],
        [ 58.8760,  68.8199, 302.0871, 299.3314]])

In [39]:
# distances

tensor([[ 61.4400,  41.2657, 271.3600, 423.3552],
        [142.3360,  45.7143,  14.3360, 356.5714],
        [ 14.3360, 165.2565, 382.9760, 109.1873],
        ...,
        [ 88.0640, 133.7658, 117.7600, 113.7778],
        [  2.0480,  94.2080, 465.9200, 330.4107],
        [206.8480,  78.6815,  -5.1200, 209.8174]])